In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")

In [3]:
test=pd.read_csv("test.csv")

In [4]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [5]:
train.corr()['pressure']

id          -0.002402
breath_id   -0.002394
R            0.015976
C           -0.036727
time_step   -0.524829
u_in         0.308136
u_out       -0.614910
pressure     1.000000
Name: pressure, dtype: float64

In [6]:
train=train.drop(['breath_id'],axis=1)
test=test.drop(['breath_id'],axis=1)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   R          int64  
 2   C          int64  
 3   time_step  float64
 4   u_in       float64
 5   u_out      int64  
 6   pressure   float64
dtypes: float64(3), int64(4)
memory usage: 322.4 MB


In [8]:
train=train.drop(['id'],axis=1)
test=test.drop(['id'],axis=1)

In [9]:
X=train.drop('pressure',axis=1)
y=train['pressure']

In [10]:
train.shape

(6036000, 6)

In [11]:
test.shape

(4024000, 5)

In [12]:
from tensorflow import keras 
from tensorflow.keras import layers


In [13]:
# model=keras.Sequential([
#     layers.Dense(1024,activation='relu',input_shape=[5]),
#     layers.Dense(512,activation='relu'),
#     layers.Dense(256,activation='sigmoid'),
#     layers.Dense(128,activation='relu'),
#     layers.Dense(64,activation='tanh'),
#     layers.Dense(32,activation='relu'),
# ])

In [14]:
#loss='mean_squared_error'
# adam
# model.compile(optimizer='adam',loss='mae')

In [15]:
X = np.array(X)
X.shape

(6036000, 5)

In [16]:
X = X.reshape(int(X.shape[0]/80), 80, X.shape[1])
X.shape

(75450, 80, 5)

In [17]:
y = np.array(y)
y.shape

(6036000,)

In [18]:
y = y.reshape(75450, 80)
y.shape

(75450, 80)

In [45]:
test.tail()

,R,C,time_step,u_in,u_out
4023995,20,10,2.530117,4.971245,1
4023996,20,10,2.563853,4.975709,1
4023997,20,10,2.597475,4.979468,1
4023998,20,10,2.631134,4.982648,1
4023999,20,10,2.665301,4.985373,1


In [49]:
from sklearn.model_selection import train_test_split

In [53]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)


In [19]:
import tensorflow as tf
norm = tf.keras.layers.Normalization(input_shape = [80, X.shape[2],], axis = -1)
norm.adapt(X)

In [20]:
m = tf.keras.Sequential([
    norm,
    tf.keras.layers.Conv1D(128, 3, activation = "relu"),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, 3, activation = "relu"),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80,)
])
m.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss= "mae")


In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

In [33]:
# batch_size=256

early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
history=m.fit(X,y,validation_data=(X,y),batch_size=512,epochs=6, callbacks=[early_stop])

Epoch 1/15
148/148 [==============================] - 394s 3s/step - loss: 0.8129 - val_loss: 0.5043
Epoch 2/15
148/148 [==============================] - 372s 3s/step - loss: 0.8111 - val_loss: 0.5041
Epoch 3/15
148/148 [==============================] - 471s 3s/step - loss: 0.8042 - val_loss: 0.5154
Epoch 4/15
148/148 [==============================] - 655s 4s/step - loss: 0.8048 - val_loss: 0.5015
Epoch 5/15
148/148 [==============================] - 622s 4s/step - loss: 0.8023 - val_loss: 0.5018
Epoch 6/15
148/148 [==============================] - 616s 4s/step - loss: 0.7962 - val_loss: 0.5015
Epoch 7/15
148/148 [==============================] - 617s 4s/step - loss: 0.7964 - val_loss: 0.4857
Epoch 8/15
148/148 [==============================] - 600s 4s/step - loss: 0.7916 - val_loss: 0.5095
Epoch 9/15
148/148 [==============================] - 605s 4s/step - loss: 0.7928 - val_loss: 0.4890
Epoch 10/15
148/148 [==============================] - 565s 4s/step - loss: 0.7895 - val_lo

KeyboardInterrupt: 

In [60]:
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 

                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
history=m.fit(X,y,validation_data=(X,y),batch_size=512,epochs=6, callbacks=[early_stop])

Epoch 1/6
148/148 [==============================] - 422s 3s/step - loss: 0.7793 - val_loss: 0.4917
Epoch 2/6
148/148 [==============================] - 583s 4s/step - loss: 0.7762 - val_loss: 0.4754
Epoch 3/6
148/148 [==============================] - 536s 4s/step - loss: 0.7755 - val_loss: 0.4763
Epoch 4/6
148/148 [==============================] - 444s 3s/step - loss: 0.7730 - val_loss: 0.4937
Epoch 5/6
148/148 [==============================] - 506s 3s/step - loss: 0.7725 - val_loss: 0.4798
Epoch 6/6
148/148 [==============================] - 604s 4s/step - loss: 0.7723 - val_loss: 0.4858


In [61]:
test=np.array(test)
test = test.reshape(int(test.shape[0]/80), 80, test.shape[1])
test.shape

(50300, 80, 5)

In [104]:
y_pred=m.predict(test)

1572/1572 [==============================] - 106s 68ms/step


In [107]:
submission=pd.read_csv('sample_submission.csv')

In [108]:
submission.head()

,id,pressure
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [109]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024000 entries, 0 to 4023999
Data columns (total 2 columns):
 #   Column    Dtype
---  ------    -----
 0   id        int64
 1   pressure  int64
dtypes: int64(2)
memory usage: 61.4 MB


In [111]:
y_pred=np.reshape(y_pred,(4024000,1))

In [112]:
submission['pressure']=y_pred

In [113]:
submission.to_csv('submission.csv')

In [114]:
submission.head()

,id,pressure
0,1,6.263421
1,2,6.092640
2,3,7.082107
3,4,7.584594
4,5,9.009420
